<a href="https://colab.research.google.com/github/rohitd3/manyFacesML/blob/master/_notebooks/CNN_usingPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
ImageWidth = 32
ImageHeight = 32

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(60),
    transforms.Resize((ImageWidth, ImageHeight)), # Modify resize to match the model's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

> Here we have several transform functions after establishing the image width and height. These transforms really limit test our model to make sure it can interact with a transformed image with how it usually does. 

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

classes = ('car', 'cat', 'deer', 'dog', 'horse', 'ship', 'truck')

> Here we establish the training set and the testing set. 

- Training is like the homework for the model. It builds the model's functionality using more data. 
- The test is like the final exam. It tests how well the model can function with the data we have given it after training.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # convert from Tensor image
    plt.show()

# training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth:', ' '.join('%5s' % classes[labels[j]] for j in range(min(4, len(labels)))))


> The main priority of this function is to help display the images through NumPy and plots. The function unnormalizes the normalized image, and converts them to be able to be displayed on a plot. 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# train_on_gpu = torch.cuda.is_available()
# print("train_on_gpu = ", train_on_gpu)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

> Here we send the model through different layers in the class Net. We send it through a total of two Convulutional layers, max pooling layers, and fully connected(linear) layers. Each layers is complex, but in total, this sends the neural network model to effectively classify and extract visual data. 

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')